Power Plant cycle
===


In [5]:
import cantera as ct
import pandas as pd
import astropy.units as u


state = pd.DataFrame({'index':['1', '2', '2a', '3', '4', '5', '5a', '6'],
                      'pressure': [None, None, None, None, None, None, None, None],
                      'temperature': [None, None, None, None, None, None, None, None],
                      'enthalpy': [None, None, None, None, None, None, None, None] 
                      }
                     )


# maximum temperature of cycle
T_max = (570 + 273.15) *u.K
P_max = 19.56e6 *u.Pa

# re-firing pressure
P_fre = 2.8e6  *u.Pa

P_reg = P_fre/6

# regeneration
T_reg = (120 + 273.15) *u.K


# ambient temperature (condensator)
T_amb = (20 + 273.15) *u.K 


fluid = ct.Water()
fluid.basis = 'mass'

#get base enthalpy at 0°C
fluid.TP = 273.16, 100325
H_base = fluid.h

# calculate the saturation pressure in the condensator
fluid.TP = T_amb.value, 100000
P_cond = fluid.P_sat *u.Pa

# 1=2
state.pressure    = [P_max, P_max, P_max, P_max, P_fre, P_fre, P_reg, P_cond]
fluid.TP = T_max.value, P_fre.value   # state 4 --> saturation Temperature
T_fre = fluid.T_sat *u.K

# 5a (regeneration)
fluid.TP = T_max.value, P_fre.value
s_reg = fluid.s
fluid.SP = s_reg, P_reg.value
T_reg = fluid.T *u.K
H_reg = fluid.h - H_base
state.enthalpy[6] = H_reg


state.temperature = [T_amb, T_amb, T_reg, T_max, T_fre, T_max, T_reg, T_amb]

# 1
fluid.TP = state.temperature[0].value, state.pressure[0].value
H_1 = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[0:1] = H_1

# 2a
fluid.TP = state.temperature[2].value, state.pressure[2].value
H_2a = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[3] = H_2a

# 3
fluid.TP = state.temperature[3].value, state.pressure[3].value
H_3 = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[4] = H_3

# 4
fluid.TP = state.temperature[4].value, state.pressure[4].value
H_4 = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[5] = H_4

# 5
fluid.TP = state.temperature[5].value, state.pressure[5].value
H_5 = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[5] = H_5

# 5a
fluid.TP = state.temperature[6].value, state.pressure[6].value
H_5a = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[6] = H_5a

# 6
fluid.TP = state.temperature[7].value, state.pressure[7].value
H_6 = (fluid.h - H_base) *u.J/u.kg
state.enthalpy[7] = H_6 


power_th = 1500e6 *u.W
eff_el = 0.4
power_el = power_th*eff_el

power_reg = 10e6 *u.W

# mass flows for regeneration 10MW
m_reg = power_reg/(H_5a-H_2a)
print('regeneration mass flow: ', m_reg.to(u.kg/u.s))

# mass flow total

from scipy.optimize import minimize_scalar
print(power_el)
m_tot = 147*u.kg/u.s

f = lambda x: (x*(H_3-H_4).value+x*(H_5-H_5a).value+ (x-m_reg.value)*(H_5a-H_6)).value-power.el
minimize_scalar(f, bounds=(m_reg.value*2, m_reg.value*5000))

print((m_tot*(H_3-H_4)+m_tot*(H_5-H_5a)+ (m_tot-m_reg)*(H_5a-H_6)).to(u.W))









ImportError: dlopen(/usr/local/lib/python3.4/site-packages/cantera/_cantera.so, 2): Library not loaded: /opt/local/lib/libsundials_cvodes.2.dylib
  Referenced from: /usr/local/lib/python3.4/site-packages/cantera/_cantera.so
  Reason: image not found